# Refer to 
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/01.train-within-notebook.ipynb
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/11.production-deploy-to-aks.ipynb

In [57]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [58]:
import azureml.core
print(azureml.core.VERSION)

0.1.65


In [59]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [60]:
#Creating the model pickle file
import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet152(weights='imagenet')

In [61]:
model

In [62]:
model_json = model.to_json()
with open("model_resnet.json", "w") as json_file:
    json_file.write(model_json)

In [63]:
model.save_weights("model_resnet_weights.h5")

In [64]:
#from keras.models import model_from_json
#model = model_from_json(model_json)

In [65]:
model.load_weights('model_resnet_weights.h5')

In [66]:
model

In [67]:
#Register the model
#from azureml.core.model import Model
#model = Model.register(model_path = "model_resnet_weights.h5", # this points to a local file
#                       model_name = "resnet_model", # this is the name the model is registered as
#                       tags = ["dl", "resnet"],
#                       description = "resnet 152 model",
#                       workspace = ws)

#print(model.name, model.description, model.version)

In [68]:
#define init() function
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')

In [69]:
init()

Model loaded


In [70]:
#define run() function 
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))

In [71]:
import wget
wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

'220px-Lynx_lynx_poing (2).jpg'

In [72]:
img_path = '220px-Lynx_lynx_poing.jpg'

In [73]:
run(img_path)

Model loaded
Predicted: [[('n02127052', 'lynx', 0.99590886), ('n02128385', 'leopard', 0.001150445), ('n02123159', 'tiger_cat', 0.0009417962)]]


In [74]:
%%writefile score.py
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')
    
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))    

Overwriting score.py


In [75]:
%run score.py

In [ ]:
!more score.py

In [ ]:
!more conda_dependencies.yml

In [76]:
import os
os.getcwd()

'/home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow'

In [77]:
os.listdir(os.getcwd())

['AML_00_configuration_v0.1.50.ipynb',
 '00_DevelopModel.ipynb',
 'AML_02_DevelopModelDriver_AKS-old.ipynb',
 'AML_01_trainlocal-docker.ipynb',
 'model_resnet_weights.h5',
 'AML_02_resnet_model_weights.ipynb',
 '01_DevelopModelDriver.ipynb',
 '__pycache__',
 'samples',
 'resnet152_n.py',
 '06_SpeedTestWebApp.ipynb',
 '03_TestLocally.ipynb',
 'AML_02_Develop_init_run_functions.ipynb',
 '220px-Lynx_lynx_poing (1).jpg',
 'AML_02_AKS_deployment_v0.1.65.ipynb',
 'AML_02_AKS_deployment.ipynb',
 'aml_config',
 '05_TestWebApp.ipynb',
 '04_DeployOnAKS.ipynb',
 'score.py',
 'model_resnet.json',
 'AML_01_trainlocal_v0.1.50.ipynb',
 'AML_00_configuration_v0.1.65.ipynb',
 '02_BuildImage.ipynb',
 '07_TearDown.ipynb',
 'helpers.py',
 'testing_utilities.py',
 '220px-Lynx_lynx_poing.jpg',
 'config',
 'conda_dependencies.yml',
 'resnet152.py',
 'AML_02_AKS_deployment_v0.1.50.ipynb',
 '220px-Lynx_lynx_poing (2).jpg',
 'README.md',
 'environment.yml',
 'sample_projects',
 'AML_01_trainlocal_v0.1.65.ipynb'

In [78]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "conda_dependencies.yml",
                                                  description = "Image for AKS Deployment Tutorial",
                                                  tags = {"name":"AKS","project":"AML"}, 
                                                  dependencies = ["resnet152.py","220px-Lynx_lynx_poing.jpg"]
                                                 )

image = ContainerImage.create(name = "myimage1",
                              # this is the model object
                              models = [],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running....................................................................
SucceededImage creation operation finished for image myimage1:9, operation "Succeeded"


In [79]:
#list images
images = ws.images()
images

{'myimage1': ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7f88dfc52828>, name=myimage1, id=myimage1:9, tags={'name': 'AKS', 'project': 'AML'}, properties={}, version=9),
 'my-aci-svc-1': ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7f88dfc52828>, name=my-aci-svc-1, id=my-aci-svc-1:4, tags={}, properties={}, version=4)}

In [ ]:
#for img in ws.images():
#    if img.name == 'myimage1': img.delete()

In [80]:
#Provision AKS cluster
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'jaya-aks-6' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [81]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating..........................................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
CPU times: user 2.82 s, sys: 135 ms, total: 2.95 s
Wall time: 16min 9s


In [82]:
!az aks get-credentials -n 'jaya-aks-6' -g jayavienna -a -f config

The Resource 'Microsoft.ContainerService/managedClusters/jaya-aks-6' under resource group 'jayavienna' was not found.


In [83]:
!kubectl --kubeconfig config get services

No resources found.
Error from server (NotAcceptable): unknown (get services)


In [84]:
#Deploy web service to AKS
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [85]:
%%time
aks_service_name ='jaya-aks-service-7'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running..................................................................
FailedAKS service creation operation finished, operation "Failed"
Service creation failed with
StatusCode: 400
Message: Kubernetes Deployment failed
Transitioning
CPU times: user 1.14 s, sys: 81.1 ms, total: 1.22 s
Wall time: 5min 43s


In [86]:
#Debug

# load workspace from default config file
ws = Workspace.from_config()

# list all web services in the workspace
#for s in ws.webservices():
#    print(s.name)

# instantiate the web service object from workspace and service name
svc = Webservice(workspace = ws, name = 'jaya-aks-service-7')

# print out the Docker log
print(svc.get_logs())

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
2018-10-05 15:21:34,697 CRIT Supervisor running as root (no user in config file)
2018-10-05 15:21:34,699 INFO supervisord started with pid 1
2018-10-05 15:21:35,701 INFO spawned: 'rsyslog' with pid 9
2018-10-05 15:21:35,702 INFO spawned: 'program_exit' with pid 10
2018-10-05 15:21:35,704 INFO spawned: 'nginx' with pid 11
2018-10-05 15:21:35,706 INFO spawned: 'iot' with pid 12
2018-10-05 15:21:35,708 INFO spawned: 'gunicorn' with pid 13
2018-10-05 15:21:35,716 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2018-10-05 15:21:35,777 INFO exited: iot (exit status 1; expected)
2018-10-05 15:21:36,779 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-10-05 15:21:36,779 INFO success: program_exit entere

In [87]:
#debug web service failure
print(ws.webservices()['jaya-aks-service-7'].get_logs())

2018-10-05 15:21:34,697 CRIT Supervisor running as root (no user in config file)
2018-10-05 15:21:34,699 INFO supervisord started with pid 1
2018-10-05 15:21:35,701 INFO spawned: 'rsyslog' with pid 9
2018-10-05 15:21:35,702 INFO spawned: 'program_exit' with pid 10
2018-10-05 15:21:35,704 INFO spawned: 'nginx' with pid 11
2018-10-05 15:21:35,706 INFO spawned: 'iot' with pid 12
2018-10-05 15:21:35,708 INFO spawned: 'gunicorn' with pid 13
2018-10-05 15:21:35,716 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2018-10-05 15:21:35,777 INFO exited: iot (exit status 1; expected)
2018-10-05 15:21:36,779 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-10-05 15:21:36,779 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-10-05 15:21:40,784 INFO success: nginx e

In [ ]:
#clean up resources
#aks_target = AksCompute(name='jaya-aks-1',workspace=ws)
#aks_target.delete()

In [ ]:
#alternate code to clean up resources
#!az aks delete --resource-group jayavienna --name jaya-aks-2 --yes

In [ ]:
#for s in ws.webservices():
#    print(s.name)

In [ ]:
#s =  Webservice(ws, 'jaya-aks-service-2')
#s.delete()

In [ ]:
#from azureml.core import Workspace
#from azureml.core.compute import AksCompute, ComputeTarget

#ws = Workspace.from_config()

#for c in ws.compute_targets():
#    print(c.name)
